In [ ]:
import numpy as np
from minst import loadData
from collections import defaultdict

trainData, trainLabel = loadData(fileName='D:/Jupyter/mnist_train.csv',label_bin=1)
testData, testLabel   = loadData(fileName='D:/Jupyter/mnist_test.csv',label_bin=1)

In [1]:
class maxEnt:
    '''
    最大熵类
    '''
    def __init__(self, trainDataList, trainLabelList, testDataList, testLabelList):
        '''
        各参数初始化
        '''
        self.trainDataList = trainDataList          #训练数据集
        self.trainLabelList = trainLabelList        #训练标签集
        self.testDataList = testDataList            #测试数据集
        self.testLabelList = testLabelList          #测试标签集
        self.featureNum = len(trainDataList[0])     #特征数量
        self.N = len(trainDataList)                 #总训练集长度
        self.n = 0                                  #训练集中（xi，y）对数量
        self.M = 10000                              #测试集长度
        self.fixy = self.calc_fixy()                #所有(x, y)对出现的次数
        self.w = [0] * self.n                       #Pw(y|x)中的w
        self.xy2idDict, self.id2xyDict = self.createSearchDict()        #(x, y)->id和id->(x, y)的搜索字典
        self.Ep_xy = self.calcEp_xy()               #Ep_xy期望值


  计算特征函数$f(x, y)$关于模型$P(Y|X)$与经验分布$\tilde{P}(X, Y)$的期望值，即“6.2.2 最大熵模型的定义”中第二个期望（83页最上方的期望）：
  $$E_{P}(f)=\sum_{x, y} \tilde{P}(x) P(y | x) f(x, y)$$

In [2]:
class maxEnt:
    def calcEpxy(self):      
        Epxy = [0] * self.n        #初始化期望存放列表，对于每一个xy对都有一个期望    

        for i in range(self.N):    #对于每一个样本进行遍历 60000            
            Pwxy = [0] * 2         #初始化公式中的P(y|x)列表  X表示是一个样本的全部特征，x表示单个特征，这里是全部特征的一个样本
            
            Pwxy[0] = self.calcPwy_x(self.trainDataList[i], 0)     #计算P(y = 0 } X)  
            Pwxy[1] = self.calcPwy_x(self.trainDataList[i], 1)     #计算P(y = 1 } X)
            
            for feature in range(self.featureNum):                 #784
                for y in range(2):                              
                    if (self.trainDataList[i][feature], y) in self.fixy[feature]:
                        id = self.xy2idDict[feature][(self.trainDataList[i][feature], y)]
                        Epxy[id] += (1 / self.N) * Pwxy[y]
        return Epxy

计算特征函数$f(x, y)$关于经验分布$\tilde{P}(X, Y)$的期望值。即“6.2.2 最大熵的定义”中第一个期望（82页最下方那个式子）：
$$
E_{\tilde{P}}(f)=\sum_{x, y} \tilde{P}(x, y) f(x, y)
$$

In [ ]:
class maxEnt:
    def calcEp_xy(self):
        Ep_xy = [0] * self.n        #初始化Ep_xy列表，长度为n

        for feature in range(self.featureNum):
            for (x, y) in self.fixy[feature]:            #遍历每个特征中的(x, y)
                id = self.xy2idDict[feature][(x, y)]                #获得其id
                Ep_xy[id] = self.fixy[feature][(x, y)] / self.N    #将计算得到的Ep_xy写入对应的位置中fixy中存放所有对在训练集中出现过的次数，处于训练集总长度N就是概率了

        return Ep_xy        #返回期望

  设置xy搜多id字典。这里的x指的是单个的特征，而不是某个样本，因此将特征存入字典时也需要存入这是第几个特征这一信息，这是为了后续的方便，否则会乱套。比如说一个样本X = (0, 1, 1) label =(1)。
生成的标签对有(0, 1), (1, 1), (1, 1)，三个(x，y)对并不能判断属于哪个特征的，后续就没法往下写。
不可能通过(1, 1)就能找到对应的id，因为对于(1, 1),字典中有多重映射。
所以在生成字典的时总共生成了特征数个字典，例如在mnist中样本有784维特征，所以生成784个字典，属于不同特征的xy存入不同特征内的字典中，使其不会混淆。

In [3]:
class maxEnt:
    def createSearchDict(self):
        '''
        创建查询字典
        xy2idDict：通过(x,y)对找到其id,所有出现过的xy对都有一个id
        id2xyDict：通过id找到对应的(x,y)对
        '''

        xy2idDict = [{} for i in range(self.featureNum)]        #初始化id到xy对的字典。[{}, {}, {}, {},……, {}, {}, {}]
        id2xyDict = {}                                          #设置索引，其实就是最后的id
        index = 0

        for feature in range(self.featureNum):
            #对出现过的每一个(x, y)对进行遍历        
            for (x, y) in self.fixy[feature]:       #fixy：内部存放特征数目个字典，对于遍历的每一个特征，单独读取对应字典内的(x, y)对
                xy2idDict[feature][(x, y)] = index  #将该(x, y)对存入字典中，要注意存入时通过[feature]指定了存入哪个特征内部的字典           
                id2xyDict[index] = (x, y)           #同时在id->xy字典中写入id号，val为(x, y)对
                index += 1

        return xy2idDict, id2xyDict

In [13]:
xy2idDict = [{} for i in range(10)]
xy2idDict[1][(3,1)]=2
print(xy2idDict)

[{}, {(3, 1): 2}, {}, {}, {}, {}, {}, {}, {}, {}]


In [16]:
id2xyDict = {}
id2xyDict[2] = (3, 1)
id2xyDict[3] = (4, 0)
print(id2xyDict)

{2: (3, 1), 3: (4, 0)}


计算(x, y)在训练集中出现过的次数

In [17]:
class maxEnt:
    def calc_fixy(self):
        fixyDict = [defaultdict(int) for i in range(self.featureNum)]#建立特征数目个字典，属于不同特征的(x, y)对存入不同的字典中
        #defaultdict(int)---字典里key不存在时返回0
        
        for i in range(len(self.trainDataList)):        
            for j in range(self.featureNum):                                            #遍历样本中所有特征            
                fixyDict[j][(self.trainDataList[i][j], self.trainLabelList[i])] += 1    #将出现过的(x, y)对放入字典中并计数值加1      
        for i in fixyDict:  
            self.n += len(i)         #对整个大字典进行计数，判断去重后还有多少(x, y)对，写入n
     
        return fixyDict              #返回大字典

In [27]:
from collections import defaultdict
fixyDict = [defaultdict(int) for i in range(10)]
fixyDict[1][(3,1)]=5
fixyDict

[defaultdict(int, {}),
 defaultdict(int, {(3, 1): 5}),
 defaultdict(int, {}),
 defaultdict(int, {}),
 defaultdict(int, {}),
 defaultdict(int, {}),
 defaultdict(int, {}),
 defaultdict(int, {}),
 defaultdict(int, {}),
 defaultdict(int, {})]

计算“6.23 最大熵模型的学习” 式6.22
$$
P_{w}(y | x)=\frac{1}{Z_{w}(x)} \exp \left(\sum_{i=1}^{n} w_{i} f_{i}(x, y)\right)
$$
$$
Z_{w}(x)=\sum_{y} \exp \left(\sum_{i=1}^{n} w_{i} f_{i}(x, y)\right)
$$

In [28]:
class maxEnt:    
    def calcPwy_x(self, X, y):
        '''
        :param X: 要计算的样本X（一个包含全部特征的样本）
        :param y: 该样本的标签
        :return: 计算得到的Pw(Y|X)
        '''
        numerator = 0        #分子
        Z = 0                #分母

        for i in range(self.featureNum):                #对每个特征进行遍历
            if (X[i], y) in self.xy2idDict[i]:          #如果该(xi,y)对在训练集中出现过            
                index = self.xy2idDict[i][(X[i], y)]    #在xy->id字典中指定当前特征i，以及(x, y)对：(X[i], y)，读取其id
                                                        #分子是wi和fi(x，y)的连乘再求和，最后指数,由于当(x, y)存在时fi(x，y)为1，因为xy对肯定存在，所以直接就是1
                                                        #对于分子来说，就是n个wi累加，最后再指数就可以了.因为有n个w，所以通过id将w与xy绑定，前文的两个搜索字典中的id就是用在这里
                numerator += self.w[index]              #同时计算其他一种标签y时候的分子，下面的z并不是全部的分母，再加上上式的分子以后才是完整的分母，即z = z + numerator

            if (X[i], 1-y) in self.xy2idDict[i]:        
                index = self.xy2idDict[i][(X[i], 1-y)]    
                Z += self.w[index]

        numerator = np.exp(numerator)        #计算分子的指数
        Z = np.exp(Z) + numerator            #计算分母的z

        return numerator / Z                #返回Pw(y|x)

In [29]:
class maxEnt:
    def maxEntropyTrain(self, iter = 500):
   
        for i in range(iter):          #设置迭代次数寻找最优解            
            Epxy = self.calcEpxy()     #计算“6.2.3 最大熵模型的学习”中的第二个期望（83页最上方哪个）            
            sigmaList = [0] * self.n   #使用的是IIS，所以设置sigma列表            
            for j in range(self.n):    #对于所有的n进行一次遍历            
                sigmaList[j] = (1 / self.M) * np.log(self.Ep_xy[j] / Epxy[j])    #依据“6.3.1 改进的迭代尺度法” 式6.34计算         
            self.w = [self.w[i] + sigmaList[i] for i in range(self.n)]           #按照算法6.1步骤二中的（b）更新w
            #单次迭代结束

In [30]:
class maxEnt:
    def predict(self, X):
        '''
        预测标签
        :param X:要预测的样本
        :return: 预测值
        ''' 
        result = [0] * 2                           #因为y只有0和1，所有建立两个长度的概率列表 
        for i in range(2):                        #循环计算两个概率      
            result[i] = self.calcPwy_x(X, i)      #计算样本x的标签为i的概率
        #返回标签
        #max(result)：找到result中最大的那个概率值
        #result.index(max(result))：通过最大的那个概率值再找到其索引，索引是0就返回0，1就返回1
        return result.index(max(result))